In [1]:
# notebook setup
import datetime as dt
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))

import brightenv2.utils.weather as weather

InteractiveShell.ast_node_interactivity = 'all'

### Get a daily weather report by latitude and longitude

In [2]:
now = dt.datetime.now()
today = dt.datetime(year=now.year, month=now.month, day=now.day); today

report = weather.weather_report(
    weather.WeatherRequest(lat=47.6062, lon=122.3321, date=today, zip_code=None)
)
pd.DataFrame(report.get('report')).head()

datetime.datetime(2018, 9, 17, 0, 0)

,apparentTemperature,cloudCover,cloudCoverError,day,dewPoint,hour,humidity,icon,lat,lon,...,temperature,temperatureError,time,uvIndex,visibility,windBearing,windBearingError,windGust,windSpeed,windSpeedError
0,45.53,0.02,None,2018-09-17,39.66,12,0.75,clear-night,47.6,122.3,...,47.04,None,2018-09-17 12:00:00,0.0,10.0,336.0,None,4.86,3.9,None


### Or get a report by zipcode

In [3]:
report = weather.weather_report(
    weather.WeatherRequest(lat=None, lon=None, date=today, zip_code=98110)
)
pd.DataFrame(report.get('report')).head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,...,hour,lat,lon,day,temperatureError,windSpeedError,windBearingError,pressureError,precipAccumulation,cloudCoverError
0,54.80,0.33,49.41,0.82,partly-cloudy-night,309.89,0.0,0.0,NaN,1019.55,...,0,47.6,-122.5,2018-09-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,54.00,0.37,49.63,0.85,partly-cloudy-night,309.81,0.0,0.0,NaN,1019.80,...,1,47.6,-122.5,2018-09-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,53.85,0.61,49.41,0.85,partly-cloudy-night,309.77,0.0,0.0,NaN,1020.09,...,2,47.6,-122.5,2018-09-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,53.29,0.97,49.49,0.87,cloudy,309.87,0.0,0.0,NaN,1020.39,...,3,47.6,-122.5,2018-09-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,52.66,0.75,48.81,0.87,partly-cloudy-night,310.15,0.0,0.0,NaN,1019.99,...,4,47.6,-122.5,2018-09-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN


### You can also request a the daily summary to be pre-computed. The DarkSky API orignally returns hourly metrics as shown in the request above. However, to return daily aggregations you can set the `summary` variable to True

In [4]:
report = weather.weather_report(
    weather.WeatherRequest(lat=None, lon=None, date=today, zip_code=98115),
    summarize=True
)
pd.DataFrame(report.get('report')).head()

,cloud_cover_IQR,cloud_cover_max,cloud_cover_mean,cloud_cover_min,cloud_cover_q25,cloud_cover_q50,cloud_cover_q75,cloud_cover_std,date,dew_point_IQR,...,summary,temp_IQR,temp_max,temp_mean,temp_min,temp_q25,temp_q50,temp_q75,temp_std,zip_code
0,0.0,0.41,0.41,0.41,0.41,0.41,0.41,NaN,2018-09-17,0.0,...,partly-cloudy-day,0.0,59.64,59.64,59.64,59.64,59.64,59.64,NaN,98115


### If you have a series of requests to make, just include a list of a requests. You can set `n_jobs` to use as many cores as you want. 
For example, the following request makes ten simultaneous calls to Dark Sky retrieving weather summaries for the past ten days.

In [7]:
report = weather.weather_report(
    [
        weather.WeatherRequest(
            lat=None, lon=None, date=today-dt.timedelta(days=i), zip_code=47579
        )
        for i in range(10)
    ],
    summarize=True,
    n_jobs=-1
)
pd.DataFrame(report.get('report')).head()

,cloud_cover_IQR,cloud_cover_max,cloud_cover_mean,cloud_cover_min,cloud_cover_q25,cloud_cover_q50,cloud_cover_q75,cloud_cover_std,date,dew_point_IQR,...,summary,temp_IQR,temp_max,temp_mean,temp_min,temp_q25,temp_q50,temp_q75,temp_std,zip_code
0,0.15,0.64,0.12,0.0,0.0,0.09,0.15,0.15,2018-09-17,1.22,...,"partly-cloudy-day, clear-night, clear-day",10.59,86.82,76.90,70.63,71.85,74.40,82.44,5.82,47579
1,0.00,0.38,0.06,0.0,0.0,0.00,0.00,0.12,2018-09-16,0.97,...,"partly-cloudy-day, clear-night, clear-day",9.82,84.80,76.65,70.07,72.00,75.00,81.82,5.28,47579
2,0.24,0.71,0.15,0.0,0.0,0.00,0.24,0.25,2018-09-15,5.40,...,"partly-cloudy-day, clear-night, clear-day",16.65,90.67,78.18,67.85,70.10,76.44,86.74,8.38,47579
3,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,2018-09-14,2.66,...,"clear-night, clear-day",18.89,88.68,76.24,65.90,67.14,73.46,86.04,8.92,47579
4,0.00,0.16,0.01,0.0,0.0,0.00,0.00,0.03,2018-09-13,4.77,...,"clear-night, clear-day",17.86,88.04,73.22,60.92,64.49,71.74,82.35,9.60,47579
